### Pipeline to Calculate Simple and Compound Interest

In [1]:
# The kfp-python package provides the Interface to Generate and Run Pipelines

import kfp
import typing

#### Read the Inputs and Pass it to the next Pipeline Block

In [2]:
# Named Tuple is used to Generate Pipeline Blocks with Multiple Output Parameters

from typing import NamedTuple
from kfp.components import *

def read_inputs(principal_amt: float, simple_interest_rate: float, compound_interest_rate: float, 
                freq_in_months: int, duration_in_months : int) -> NamedTuple('Outputs', [('principal', float), 
                                                                                         ('si', float), 
                                                               ('ci', float), ('freq', int), ('duration',int)]):
    
    return (principal_amt, simple_interest_rate, compound_interest_rate, freq_in_months, duration_in_months)


In [3]:
kfp_read_inputs = kfp.components.func_to_container_op(func = read_inputs, 
                                                          output_component_file = './read_inp.yaml')


#### Calculate Simple Interest

In [4]:
def si_calc(p : float, si_r : float, fr : int, dur: int)-> NamedTuple('S_outs', [('pr', float), ('si_r', float), 
                                                               ('freq', int), ('dur',int), ('amt', float)]):
    amt = p + (p*(si_r/100)*(int(dur/fr)))
    amt = "{:.2f}".format(amt)
    return (p, si_r, fr, dur, amt)

In [5]:
kfp_si = kfp.components.func_to_container_op(func = si_calc, 
                                                          output_component_file = './si_calc.yaml')


#### Print and Display

In [6]:
def si_disp(p : float, si_r : float, fr : int, dur: int, amt: float)-> str:
    s_out = "Principal Amount : {pr}    \nSimple Interest Rate : {si}% for every {frq} months   \nTotal Time Duration : {durn} months  \nFinal Amount : {amount}".format(pr = p,
    si = si_r, frq = fr, durn = dur, amount = amt)
    print(s_out)
    return s_out
    

In [7]:
kfp_si_disp = kfp.components.func_to_container_op(func = si_disp, 
                                                          output_component_file = './si_disp.yaml')


#### Calculate Compund Interest

In [8]:
def ci_calc(p : float, ci_r : float, fr : int, dur: int)-> NamedTuple('S_outs', [('pr', float), ('ci_r', float), 
                                                               ('freq', int), ('dur',int), ('amt_c', float)]):
    amt_c = p*(pow((1+ci_r/100), int(dur/fr)))
    amt_c = "{:.2f}".format(amt_c)
    return (p, ci_r, fr, dur, amt_c)

In [9]:
kfp_ci = kfp.components.func_to_container_op(func = ci_calc, 
                                                          output_component_file = './ci_calc.yaml')


#### Display

In [10]:
def ci_disp(p : float, ci_r : float, fr : int, dur: int, amt_c: float)-> str:
    s_out = "Principal Amount : {pr}\t\t   \nCompound Interest Rate : {si}% compounded every {frq} months   \nTotal Time Duration : {durn} months  \nFinal Amount : {amount}".format(pr = p,
    si = ci_r, frq = fr, durn = dur, amount = amt_c)
    print(s_out)
    return s_out
    

In [11]:
kfp_ci_disp = kfp.components.func_to_container_op(func = ci_disp, 
                                                          output_component_file = './si_disp.yaml')


#### Defining the Pipeline Execution Sequence and Input-Output scheme

In [12]:
import kfp.dsl as dsl

@dsl.pipeline(name='SI-CI Pipeline',description='Calculates Simple and Compund Interest')
def Interest_func(principal_amt: float, simple_interest_rate: float, compound_interest_rate: float, 
                freq_in_months: int, duration_in_months : int):
    
    #Passing pipeline parameter and a constant value as operation arguments
    read_input_task = kfp_read_inputs(principal_amt, simple_interest_rate, compound_interest_rate, 
                freq_in_months, duration_in_months) 
    si_calc_task = kfp_si(read_input_task.outputs['principal'], read_input_task.outputs['si'], 
                          read_input_task.outputs['freq'], read_input_task.outputs['duration'])
    si_disp_task = kfp_si_disp(si_calc_task.outputs['pr'], si_calc_task.outputs['si_r'], 
                          si_calc_task.outputs['freq'], si_calc_task.outputs['dur'], si_calc_task.outputs['amt'])
    ci_calc_task = kfp_ci(read_input_task.outputs['principal'], read_input_task.outputs['ci'], 
                          read_input_task.outputs['freq'], read_input_task.outputs['duration'])
    ci_disp_task = kfp_ci_disp(ci_calc_task.outputs['pr'], ci_calc_task.outputs['ci_r'], 
                          ci_calc_task.outputs['freq'], ci_calc_task.outputs['dur'], ci_calc_task.outputs['amt_c'])
    

#For an operation with a single return value, the output reference can be accessed using `task.output` or `task.outputs['output_name']` syntax
#For an operation with a multiple return values, the output references can be accessed using `task.outputs['output_name']` syntax

#### Compiling the Pipeline

In [13]:
pipeline_func = Interest_func
pipeline_filename = pipeline_func.__name__+'.pipeline.tar.gz'

import kfp.compiler as comp
comp.Compiler().compile(pipeline_func, pipeline_filename)